In [5]:


#!/opt/conda/bin/python3

import tempfile
from typing import Dict

import click
import toml
from snowflake.snowpark import Session
from typing import List
from evaluate import load

from audio2text.data import get_dataset
from audio2text.models import openai_whisper, nemo_canary
from audio2text.models.api import InputRow, OutputRow
from audio2text.models.api import InputRow, OutputRow
from nemo.collections.asr.models import EncDecMultiTaskModel

from audio2text.utils import (
    init_logger,
    get_rank,
    get_world_size,
    create_session,
)

logger = init_logger(__name__)


local_dir="/Users/aivanou/code/spcs-templates/unstructured_data/audio2text/data"

model_name='nemo-canary'
# config = toml.load('./configs/openai-whisper-tiny.toml')
# model = get_model(model_name, config)

dataset = get_dataset(local_dir)

batch_records = dataset[0:10]

input_batch = []
for idx in range(len(batch_records['audio_id'])):
    input_batch.append(InputRow(
        audio_id=batch_records['audio_id'][idx],
        text=batch_records['raw_text'][idx],
        filepath=batch_records['audio'][idx]['path'],
        audio_data=batch_records['audio'][idx]['array'],
    ))







In [9]:


model = EncDecMultiTaskModel.from_pretrained('nvidia/canary-1b')
model = model.eval()



In [11]:

input_files = [row.filepath for row in input_batch]

batch_data = [input_batch[0].audio_data, input_batch[1].audio_data]

transcriptions = model.transcribe(
    batch_data,
    batch_size=4,  # batch size to run the inference with
    task="ast",  # use "ast" for speech-to-text translation
    source_lang="en",  # language of the audio input, set `source_lang`==`target_lang` for ASR
    target_lang="en",  # language of the text output
    pnc='True',
)



In [12]:

transcriptions
